# **📑 Final Project Notebook**

## 1. 환경 (Scene) 구성

*   Ground / Water Plane 생성
*   위치, 스케일, 높이 설정



In [ ]:
# Isaac Sim 및 USD(Stage) 관련 유틸 함수와 클래스들을 임포트
from omni.isaac.core.utils.stage import add_reference_to_stage
from isaacsim.core.api import World
import omni.usd
from omni.isaac.core.prims import XFormPrim
from pxr import Sdf, Gf, Usd, UsdGeom, UsdShade, Vt
import omni.isaac.core.utils.stage as stage_utils # add_reference_to_stage / add_ground_plane
import omni.kit.commands

# Stage 핸들 가져오기 (항상 제일 먼저 실행)
stage = omni.usd.get_context().get_stage()

In [ ]:
from pxr import UsdGeom, Gf

# create Ground
ground = UsdGeom.Cube.Define(stage, "/World/Ground")
ground.AddTranslateOp().Set(Gf.Vec3f(0, 0, -0.1))
ground.AddScaleOp().Set(Gf.Vec3f(200, 200, 0.1))

# create Water
water = UsdGeom.Cube.Define(stage, "/World/Water")
water.AddTranslateOp().Set(Gf.Vec3f(0, 0, 1.2))
water.AddScaleOp().Set(Gf.Vec3f(200, 200, 0.05))


In [ ]:
# Make ground and water

AREA = 200.0   # 지면 크기
GROUND_Z = 0.0
WATER_Z = 1.2 #water height = 1.2m

ground = UsdGeom.Plane.Define(stage, "/World/Ground")
water = UsdGeom.Plane.Define(stage, "/World/Water")

In [ ]:
# Ground(지면) Transform 옵션 설정

ground.AddTranslateOp().Set(Gf.Vec3f(0, 0, GROUND_Z))
ground.AddScaleOp().Set(Gf.Vec3f(AREA*0.5, AREA*0.5, 1.0))
ground.AddRotateXOp().Set(0.0)
ground.AddRotateYOp().Set(0.0)
ground.AddRotateZOp().Set(0.0)

# Water(물) Transform 옵션 설정
water.AddTranslateOp().Set(Gf.Vec3f(0, 0, WATER_Z))
water.AddScaleOp().Set(Gf.Vec3f(AREA*0.5, AREA*0.5, 1.0))
water.AddRotateXOp().Set(0.0)
water.AddRotateYOp().Set(0.0)
water.AddRotateZOp().Set(0.0)

ErrorException: 
	Error in 'pxrInternal_v0_22__pxrReserved__::UsdGeomXformable::AddXformOp' at line 190 in file /builds/omniverse/usd-ci/USD/pxr/usd/usdGeom/xformable.cpp : 'The xformOp 'xformOp:translate' already exists in xformOpOrder [[xformOp:translate, xformOp:scale]].'

## 조명 (Lighting)


*   Sun light 구성
*   SkyDome 환경 조명 설정



In [ ]:
from pxr import UsdLux

# Sun(DistantLight) 정의
sun = UsdLux.DistantLight.Define(stage, "/World/Sun")

# 광원 속성 설정
sun.CreateIntensityAttr(30000.0)   # 매우 강한 조명
sun.CreateAngleAttr(0.53)           # 빛 확산 각도 설정

# 방향: -Z (위에서 아래로 직광)
sun.AddRotateXOp().Set(-90.0)
sun.AddRotateYOp().Set(0.0)
sun.AddRotateZOp().Set(0.0)


In [ ]:
# --- 하늘돔 (SkyDome) ---
mat_path = "/World/Looks/Vintage_Sky_Blue"
mat_prim = stage.GetPrimAtPath(mat_path)

In [ ]:
from pxr import UsdLux, Gf
import omni.usd

stage = omni.usd.get_context().get_stage()

# DomeLight → 약하게 (환경광 역할)
dome = stage.GetPrimAtPath("/World/DomeLight")
if dome:
    dome_light = UsdLux.DomeLight(dome)
    dome_light.CreateIntensityAttr(800.0)     # 기존 1200 → 800
    dome_light.CreateExposureAttr(0.0)        # 노출 기본
    dome_light.CreateColorAttr(Gf.Vec3f(0.95, 0.97, 1.0))

# Sun → 메인광, 조금만 줄임
sun = stage.GetPrimAtPath("/World/Sun")
if sun:
    sun_light = UsdLux.DistantLight(sun)
    sun_light.CreateIntensityAttr(2200.0)   # 기존 3000 → 2200
    sun_light.CreateAngleAttr(0.6)          # 퍼짐 조금 확대 (그림자 덜 날카롭게)


## Material 적용


*   Ground, Water, Object에 Material 바인딩





In [ ]:
from pxr import UsdGeom, Gf

stage.RemovePrim("/World/Ground")
stage.RemovePrim("/World/Water")

ground = UsdGeom.Cube.Define(stage, "/World/GroundCube")
ground.AddTranslateOp().Set(Gf.Vec3f(0, 0, -0.1))
ground.AddScaleOp().Set(Gf.Vec3f(200, 200, 0.1))

water = UsdGeom.Cube.Define(stage, "/World/WaterCube")
water.AddTranslateOp().Set(Gf.Vec3f(0, 0, 1.2))
water.AddScaleOp().Set(Gf.Vec3f(200, 200, 0.05))


In [ ]:
from pxr import UsdShade

# GroundCube 머티리얼 바인딩
ground = stage.GetPrimAtPath("/World/GroundCube")
ground_mat_prim = stage.GetPrimAtPath("/World/Looks/Ground_Aggregate_Charred_Moss")
if ground_mat_prim.IsValid():
    ground_mat = UsdShade.Material(ground_mat_prim)
    UsdShade.MaterialBindingAPI(ground).Bind(ground_mat)

# WaterCube 머티리얼 바인딩
water = stage.GetPrimAtPath("/World/WaterCube")
water_mat_prim = stage.GetPrimAtPath("/World/Looks/Water_Ocean_Light_Blue")
if water_mat_prim.IsValid():
    water_mat = UsdShade.Material(water_mat_prim)
    UsdShade.MaterialBindingAPI(water).Bind(water_mat)




In [ ]:
# box 객체에 랜덤한 Material 적용하기

import random
from pxr import Usd, UsdShade

# 사용할 머터리얼 후보 3개
materials = [
    "/World/Looks/ABS_Hard_Leather_Vintage_Rose",
    "/World/Looks/ABS_Puffy_Woven_Leather_Ash",
    "/World/Looks/ABS_Puffy_Woven_Leather_Brown"
]

# Stage 불러오기
stage = omni.usd.get_context().get_stage()

# box2_* 로 시작하는 모든 prim 찾기
for i in range(10):  # box2_0 ~ box2_9
    mesh_path = f"/World/box2_{i}/SM_FlatBox_A01_Body_01"  # 실제 메쉬 경로
    mesh_prim = stage.GetPrimAtPath(mesh_path)
    if not mesh_prim.IsValid():
        continue

    # 랜덤 머터리얼 선택
    mat_path = random.choice(materials)
    mat_prim = stage.GetPrimAtPath(mat_path)
    if not mat_prim.IsValid():
        continue

    mat = UsdShade.Material(mat_prim)
    UsdShade.MaterialBindingAPI(mesh_prim).Bind(mat)

    print(f"box2_{i} → {mat_path} 적용 완료 ✅")


box2_0 → /World/Looks/ABS_Puffy_Woven_Leather_Ash 적용 완료 ✅
box2_1 → /World/Looks/ABS_Hard_Leather_Vintage_Rose 적용 완료 ✅
box2_2 → /World/Looks/ABS_Puffy_Woven_Leather_Brown 적용 완료 ✅
box2_3 → /World/Looks/ABS_Puffy_Woven_Leather_Ash 적용 완료 ✅
box2_4 → /World/Looks/ABS_Puffy_Woven_Leather_Brown 적용 완료 ✅
box2_5 → /World/Looks/ABS_Hard_Leather_Vintage_Rose 적용 완료 ✅
box2_6 → /World/Looks/ABS_Hard_Leather_Vintage_Rose 적용 완료 ✅
box2_7 → /World/Looks/ABS_Hard_Leather_Vintage_Rose 적용 완료 ✅
box2_8 → /World/Looks/ABS_Puffy_Woven_Leather_Brown 적용 완료 ✅
box2_9 → /World/Looks/ABS_Hard_Leather_Vintage_Rose 적용 완료 ✅

In [ ]:
# pail에 meterial 적용

from pxr import UsdShade

# Mesh 프림 경로 (pail1 안쪽 Mesh)
pail_mesh_prim = stage.GetPrimAtPath("/World/pail1/SM_PlasticPail_A03_01")
mat_prim       = stage.GetPrimAtPath("/World/Looks/Aging_Copper")

# Material 객체
material = UsdShade.Material(mat_prim)

# Mesh 프림에 직접 바인딩
UsdShade.MaterialBindingAPI(pail_mesh_prim).Bind(material)

print("✅ pail1 Mesh에 Aging_Copper 머터리얼 적용 완료")


✅ pail1 Mesh에 Aging_Copper 머터리얼 적용 완료

In [ ]:
from pxr import Usd, UsdShade, Sdf
import omni.usd

stage = omni.usd.get_context().get_stage()

# ---------------------------
# ① Mesh Prim (물)
# ---------------------------
water_prim_path = "/World/Water"
water_prim = stage.GetPrimAtPath(water_prim_path)

# ---------------------------
# ② Material Prim (머티리얼)
# ---------------------------
material_path = "/World/Looks/Water_Murky_Perlinwaves"
material_prim = stage.GetPrimAtPath(material_path)

if material_prim.IsValid() and water_prim.IsValid():
    # 머티리얼 객체 생성
    material = UsdShade.Material(material_prim)

    # 물 Mesh에 머티리얼 바인딩
    UsdShade.MaterialBindingAPI(water_prim).Bind(material)
    print(f"✅ 머티리얼({material_path})을 {water_prim_path}에 적용 완료")

    # ---------------------------
    # ③ Shader 속성 수정
    # ---------------------------
    for child in material_prim.GetChildren():
        if child.GetTypeName() == "Shader":
            shader = UsdShade.Shader(child)

            # Appearance 관련 속성
            shader.CreateInput("water_tint", Sdf.ValueTypeNames.Float).Set(0.9)        # 갈색/녹색 톤
            shader.CreateInput("water_absorption", Sdf.ValueTypeNames.Float).Set(0.8)  # 더 탁하게
            shader.CreateInput("scattering_tint", Sdf.ValueTypeNames.Float).Set(0.5)   # 산란 색 낮춤
            shader.CreateInput("water_scattering_amount", Sdf.ValueTypeNames.Float).Set(0.2)  # 산란 강도 확 줄임

            # Procedural Waves (파도)
            shader.CreateInput("wave_amplitude", Sdf.ValueTypeNames.Float).Set(0.25)   # 파도 높이
            shader.CreateInput("wave_frequency", Sdf.ValueTypeNames.Float).Set(2.5)    # 파도 빈도

            print("✅ Water Material 속성 업데이트 완료")

else:
    print("경로를 다시 확인하세요 (Water 또는 Material Prim 없음)")


✅ /World/Looks/Water_Murky_Perlinwaves 머티리얼을 /World/Water에 적용 완료

## Asset 배치


*   사람, 자동차, 집 등 마을 구성 요소
*   Logs, Construction Debirs, box 같은 부가 요소




In [ ]:
# 사람 모델 불러오기
from omni.kit.commands import execute
import omni.usd

# Stage 불러오기
stage = omni.usd.get_context().get_stage()

def load_fbx(fbx_path, prim_path):
    """FBX 파일을 불러오는 함수"""
    execute("CreateReferenceCommand",
            usd_context=omni.usd.get_context(),
            path_to=prim_path,
            asset_path=fbx_path)
    print(f"✅ FBX 모델 불러오기 완료: {prim_path}")


# 불러올 FBX 파일 경로와 Prim Path 정의
fbx_models = [
    {
        "fbx_path": "/home/nvidia/Omniverse/Assets/renderpeople_free_rigged_people_FBX/carla/rp_carla_rigged_001_zup_a.fbx",
        "prim_path": "/World/rp_carla_rigged_001_zup_a"
    },
    {
        "fbx_path": "/home/nvidia/Omniverse/Assets/renderpeople_free_rigged_people_FBX/eric/rp_eric_rigged_001_zup_a.fbx",
        "prim_path": "/World/rp_eric_rigged_001_zup_a"
    }
]

# FBX 불러오기 실행
for model in fbx_models:
    load_fbx(model["fbx_path"], model["prim_path"])


In [ ]:
# box 객체 10개를 랜덤한 위치로 불러오기
import random
from omni.kit.commands import execute
import omni.usd
from pxr import UsdGeom, Gf

stage = omni.usd.get_context().get_stage()

def create_and_scatter_boxes(base_name, usd_path, count=5, water_height=1.2):
    for i in range(count):
        prim_path = f"/World/{base_name}_{i}"
        execute("CreateReferenceCommand",
                usd_context=omni.usd.get_context(),
                path_to=prim_path,
                asset_path=usd_path)

        prim = stage.GetPrimAtPath(prim_path)
        if not prim.IsValid():
            print(f"❌ {prim_path} 생성 실패")
            continue

        xform = UsdGeom.Xformable(prim)

        # 이미 존재하는 오퍼레이터 찾아오기
        ops = {op.GetOpName(): op for op in xform.GetOrderedXformOps()}

        # 랜덤 좌표/스케일/회전
        tx = random.uniform(-10, 10)
        ty = random.uniform(-10, 10)
        tz = water_height
        scale = random.uniform(1, 3)
        rot = random.uniform(0, 360)

        # translate
        if "xformOp:translate" in ops:
            ops["xformOp:translate"].Set(Gf.Vec3f(tx, ty, tz))
        else:
            xform.AddTranslateOp().Set(Gf.Vec3f(tx, ty, tz))

        # scale
        if "xformOp:scale" in ops:
            ops["xformOp:scale"].Set(Gf.Vec3f(scale, scale, scale))
        else:
            xform.AddScaleOp().Set(Gf.Vec3f(scale, scale, scale))

        # rotateZ
        if "xformOp:rotateZ" in ops:
            ops["xformOp:rotateZ"].Set(rot)
        else:
            xform.AddRotateZOp().Set(rot)

        print(f"✅ {prim_path} → pos=({tx:.2f}, {ty:.2f}, {tz}), scale={scale:.2f}, rotZ={rot:.1f}")

# 실행 예시
box2_path = "/home/nvidia/Downloads/box2.usd"
create_and_scatter_boxes("box2", box2_path, count=10, water_height=1.0)


✅ /World/box2_0 → pos=(0.55, -2.38, 1.0), scale=1.66, rotZ=147.9
✅ /World/box2_1 → pos=(-8.87, -5.34, 1.0), scale=1.55, rotZ=4.3
✅ /World/box2_2 → pos=(9.18, -9.03, 1.0), scale=1.01, rotZ=167.5
✅ /World/box2_3 → pos=(1.89, -6.15, 1.0), scale=2.39, rotZ=168.8
✅ /World/box2_4 → pos=(-4.17, 6.66, 1.0), scale=2.14, rotZ=198.5
✅ /World/box2_5 → pos=(4.06, -5.47, 1.0), scale=1.38, rotZ=163.8
✅ /World/box2_6 → pos=(1.98, -1.30, 1.0), scale=2.69, rotZ=45.5
✅ /World/box2_7 → pos=(-3.06, -7.72, 1.0), scale=2.23, rotZ=270.4
✅ /World/box2_8 → pos=(6.22, 2.94, 1.0), scale=2.03, rotZ=13.5
✅ /World/box2_9 → pos=(-3.91, 7.27, 1.0), scale=2.35, rotZ=350.4

In [ ]:
# pail, tree 불러오기

from omni.kit.commands import execute
import omni.usd
from pxr import UsdGeom, Gf

stage = omni.usd.get_context().get_stage()

def create_and_place_asset(name, usd_path, scale=0.1, pos=(0,0), height=1.2, rot=(0,0,0)):
    """프림 생성 + 참조 + 변환 세팅"""
    execute("CreateReferenceCommand",
            usd_context=omni.usd.get_context(),
            path_to=f"/World/{name}",
            asset_path=usd_path)

    prim = stage.GetPrimAtPath(f"/World/{name}")
    if not prim.IsValid():
        print(f"❌ {name} 생성 실패")
        return

    xform = UsdGeom.XformCommonAPI(prim)
    xform.SetScale(Gf.Vec3f(scale, scale, scale))          # 크기 조정
    xform.SetTranslate(Gf.Vec3d(pos[0], pos[1], height))   # 위치
    xform.SetRotate(Gf.Vec3f(*rot))                        # 회전

    print(f"✅ {name} 생성 및 배치 완료")

# 경로 (Downloads 폴더 기준)
pail_path = "/home/nvidia/Downloads/pail1.usd"


# 두 개 생성
create_and_place_asset("pail1", pail_path, scale=0.05, pos=(1, -2), height=1.2)


✅ pail1 생성 및 배치 완료
✅ tree4 생성 및 배치 완료

In [ ]:
# 자동차 불러오기

car_path = "/World/car1"
car_usdz = "/home/nvidia/Downloads/car1.usdz"

# USD Stage에 오브젝트 생성 + 참조
execute("CreateReferenceCommand",
        usd_context=omni.usd.get_context(),
        path_to=car_path,
        asset_path=car_usdz)

# 크기 조정 (40,40,40)
car_prim = stage.GetPrimAtPath(car_path)
if car_prim.IsValid():
    xform = car_prim.GetAttribute("xformOp:scale")
    if not xform.IsValid():
        car_prim.CreateAttribute("xformOp:scale", omni.usd.common.GetTypeFromAttr("float3"))
    car_prim.GetAttribute("xformOp:scale").Set((40.0, 40.0, 40.0))
    print("✅ 자동차 불러오기 + 크기 조정 완료")
else:
    print("❌ Prim 생성 실패:", car_path)

✅ 자동차 불러오기 + 크기 조정 완료

In [ ]:
# 집 (house) 불러오기

from omni.kit.commands import execute
import omni.usd
from pxr import UsdGeom, Gf

stage = omni.usd.get_context().get_stage()

# 불러올 파일 경로
usd_path = "/home/nvidia/Downloads/house.usdz"
prim_path = "/World/House2"

# 프림 생성 + 참조
execute("CreateReferenceCommand",
        usd_context=omni.usd.get_context(),
        path_to=prim_path,
        asset_path=usd_path)

# 위치 (10,0,0)에 배치
prim = stage.GetPrimAtPath(prim_path)
UsdGeom.XformCommonAPI(prim).SetTranslate(Gf.Vec3f(10, 0, 0))


ArgumentError: Python argument types in
    XformCommonAPI.SetTranslate(XformCommonAPI, Vec3f)
did not match C++ signature:
    SetTranslate(pxrInternal_v0_22__pxrReserved__::UsdGeomXformCommonAPI {lvalue}, pxrInternal_v0_22__pxrReserved__::GfVec3d translation, pxrInternal_v0_22__pxrReserved__::UsdTimeCode time=Usd.TimeCode.Default())

In [ ]:
# 통나무 불러오기

stage = omni.usd.get_context().get_stage()

# 1. Logs.usdz 불러오기
execute("CreateReferenceCommand",
        usd_context=omni.usd.get_context(),
        path_to="/World/Logs",
        asset_path="/home/nvidia/Downloads/Logs.usdz")

# 2. Construction_Debris.usdz 불러오기
execute("CreateReferenceCommand",
        usd_context=omni.usd.get_context(),
        path_to="/World/Construction_Debris",
        asset_path="/home/nvidia/Downloads/Construction_Debris.usdz")

print("✅ Logs + Construction Debris 임포트 완료")

✅ Logs + Construction Debris 임포트 완료